In [ ]:
import nltk
import os
import re
from collections import OrderedDict
from bs4 import BeautifulSoup
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
def read_folder(folder_path, base_id):
    
    snow_stemmer = SnowballStemmer(language='english')
    stop_list = stopwords.words("english")
    tokens = OrderedDict()
    nltk.download('words')

    i = 0
    for filename in os.listdir(folder_path):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8', errors='ignore') as f:
            pos = 1
            soup = BeautifulSoup(f.read(), "html.parser")
            text = soup.body.get_text()
            if text is not None:
                text_toks = nltk.RegexpTokenizer(r'\w+').tokenize(text.lower())
                file_dict = {}
                for token in text_toks:
                    stem = snow_stemmer.stem(token).lower()
                    if stem not in stop_list:
                        if stem not in file_dict.keys():
                            file_dict[stem] = []
                        file_dict[stem].append(pos)
                    pos += 1
                for key, value in file_dict.items():
                    if key not in tokens.keys():
                        tokens[key] = {}
                    tokens[key][base_id] = value
                base_id += 1
            i += 1
            if i > 5:
                break

    return OrderedDict(sorted(tokens.items()))

In [ ]:
def save_inverted_index(index, id):

    f_terms = open('index_' + str(id) + '_terms.txt', 'w')
    f_posting = open('index_' + str(id) + '_postings.txt', 'w')
    
    offset = 0
    for key, value in index.items():
        out = str(len(value))
        prev_file = 0
        for k, v in value.items():
            out += ' ' + str(k-prev_file)
            out += ' ' + str(len(v))
            prev_pos = 0
            for pos in v:
                out += ' ' + str(pos-prev_pos)
                prev_pos = pos                
            prev_file = k
        out += '\n'

        try:
            f_terms.write(key + ' ' + str(offset) + '\n')
            f_posting.write(out)
        except:
            pass

        offset += len(out) + 1

    f_terms.close()
    f_posting.close()

In [ ]:
def display_dict(dict):
    for r, y in dict.items():
        print(r, y)

In [ ]:
def search(term, id):
  stem = SnowballStemmer(language='english').stem(term).lower()

  print(stem)

  offset = -1
  with open('index_' + str(id) + '_terms.txt') as f_terms:
      for line in f_terms:
          k, off = line.split(' ')
          if term == k:
              offset = int(off)
              break

  if offset == -1:
      print("Not Found")
      return

  posting = ''
  with open('index_' + str(id) + '_postings.txt') as f_posting:
      f_posting.seek(offset)
      posting = f_posting.readline()


  postings = posting.split(' ')
  postings = list(map(int, postings))


  prev_file = 0
  x = 1
  for i in range(postings[0]):
      file_id = prev_file + postings[x]
      x += 1
      print(file_id, ':', end='\t')
      prev_off = 0
      for j in range(postings[x]):
          x += 1
          offset = prev_off + postings[x]
          print(offset, end=' ')
          prev_off = offset

      x += 1
      print()
      prev_file = file_id



In [ ]:
nltk.download('stopwords')
base_ids = [1, 1162, 2207]
choice = int(input("Enter 1, 2, 3 to select folder:"))

dic = read_folder("corpus1/" + str(choice), base_ids[choice-1])
# display_dict(dic)
save_inverted_index(dic, choice)
search('fine', choice)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Enter 1, 2, 3 to select folder:1
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
fine
2 :	2 4 
4 :	2 4 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True